# Strategy in Finding a Business Location

To find the best business location, start by analyzing your target market, researching potential areas, and considering factors like accessibility, infrastructure, and zoning regulations.
Here's a more detailed strategy:  

##  1. Define Your Needs and Goals: 
- **Target Market:** Identify your ideal customer base and where they are located
    - For definition see [this](https://www.investopedia.com/terms/t/target-market.asp#:~:text=Demographic%3A%20These%20are%20the%20main,in%20the%20era%20of%20globalization.)
- **Business Type:** Determine if your business requires 
    - high foot traffic
    - proximity to suppliers
    - access to specific infrastructure (e.g., highways, rail yards)
- **Budget:** Establish a realistic budget for 
    - rent
    - utilities
    - other location-related costs
- **Brand Image:** Consider how the location will impact your brand's image and customer perception

### Market Analysis
Example: Laundromat in Vandergrift, PA

- **Demographic Analysis:** Use local census data to understand the demographics of your area. 
    - Look for age distributions, income levels, and household sizes that match the typical profile of a user (for our example laundromat user).
- **Psychographic Information:** Go beyond basic demographics to explore customer lifestyles, values, and habits. 

For instance, are there large groups of environmentally conscious consumers in your area who might appreciate eco-friendly laundry solutions?

In [ ]:
# Import modules
import matplotlib.pyplot as plt
import pandas as pd
from census import Census
from us import states
import requests
from typing import Dict, Optional
import json
from pathlib import Path
import zipcodes
import addfips


# Created modules
from ipython_config import CENSUS_KEY


In [ ]:
# Set API key
c = Census(CENSUS_KEY)

In [ ]:
# Coin-Operated Laundries and Drycleaners for all years!
NAICS = '812310'

# Vandergrift, PA
ZIPCODE = '15690'

state = states.PA
STATEFIPS = state.fips
STATENAME = state.name

result = zipcodes.matching(ZIPCODE)[0]
COUNTYNAME = result['county']

af = addfips.AddFIPS()

# Get FIPS code for a single county
COUNTYFIP = af.get_county_fips(county=COUNTYNAME, state=STATENAME)
COUNTYCODE = COUNTYFIP[2:]

print(f"""
NAICS: {NAICS}
ZIPCODE: {ZIPCODE}
STATEFIPS: {STATEFIPS}
STATENAME: {STATENAME}
COUNTYNAME: {COUNTYNAME} 
COUNTYFIP: {COUNTYFIP}  
COUNTYCODE: {COUNTYCODE}
""")

## Locate & Analyze Customers and Market with **Census Business Builder**
- [Video How-To](https://www.census.gov/data/academy/data-gems/2023/locate-analyze-customers-market-with-cbb.html)
- [Census Business Builder](https://cbb.census.gov/cbb/)

In [ ]:
for key, value in result.items():
    print(f'{key.replace('_', '').upper()}: {value}')

### Poverty Rate by Tract (County and State specified)

In [ ]:
# Obtain Census variables from the 2019 ACS at the tract level for the Commonwealth of Virginia (FIPS code: 51)
# C17002_001E: count of ratio of income to poverty in the past 12 months (total)
# C17002_002E: count of ratio of income to poverty in the past 12 months (< 0.50)
# C17002_003E: count of ratio of income to poverty in the past 12 months (0.50 - 0.99)
# B01003_001E: total population
# Sources: https://api.census.gov/data/2019/acs/acs5/variables.html; https://pypi.org/project/census/
census = c.acs5.state_county_tract(fields = ('NAME', 'C17002_001E', 'C17002_002E', 'C17002_003E', 'B01003_001E'),
                                      state_fips = STATEFIPS,
                                      county_fips = COUNTYCODE,
                                      tract = "*",
                                      year = 2022)

In [ ]:
# Create a dataframe from the census data
df = pd.DataFrame(census)

# Show the dataframe
print('Shape: ', df.shape)

In [ ]:
df.head()

In [ ]:
# Combine state, county, and tract columns together to create a new string and assign to new column
df["GEOID"] = df["state"] + df["county"] + df["tract"]

In [ ]:
# Remove columns
df = df.drop(columns = ["state", "county", "tract"])

# Show updated dataframe
df.head(2)

In [ ]:
# Check column data types for census data
print("Column data types for census data:\n{}".format(df.dtypes))

# Source: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dtypes.html

In [ ]:
# Get poverty rate and store values in new column
df["Poverty_Rate"] = (df["C17002_002E"] + df["C17002_003E"]) / df["B01003_001E"] * 100

# Show dataframe
df[['NAME', 'Poverty_Rate']].head()

## API's For Small Business Statistics
- Nonemployer Statistics
- Business Patterns County Business Patterns
- Economic Census

## Annual Business Survey (ABS) Data

In [ ]:
# Company Summary

nonemp_params = {
    'variables' : "NAME,CBSA,COUNTY,CSA,NAICS2022,NAICS2022_LABEL,NESTAB,NRCPTOT,NRCPTOT_N,RCPSZES,YEAR",
    'geography': f"county:{COUNTYCODE}&in=state:{STATEFIPS}",
    'api_key': CENSUS_KEY,
    'dataset_base': 'nonemp',
    'year': '2022'
}

In [ ]:
# Characteristics of Businesses

cbp_params = {
    'variables' : "NAME,CBSA,CD,COUNTY,CSA,EMP,EMP_N,EMPSZES_LABEL,ESTAB,NAICS2017,NAICS2017_LABEL,PAYANN,PAYANN_N,PAYQTR1,PAYQTR1_N,YEAR,ZIPCODE",
    'geography': f"county:{COUNTYCODE}&in=state:{STATEFIPS}",
    'api_key': CENSUS_KEY,
    'dataset_base': 'cbp',
    'year': '2022'
}


In [ ]:

# Characteristics of Business Owners
ecnbasic_params = {
    'variables' : "NAME,CONCIT,COUNTY,CSA,CSLDL,NAICS2022,NAICS2022_LABEL,ECPLACE,EMP,EMP_IMP,FIRM,PAYANN,PAYQTR1,YEAR",
    'geography': f"county:{COUNTYCODE}&in=state:{STATEFIPS}",
    'api_key': CENSUS_KEY,
    'dataset_base': 'ecnbasic',
    'year': '2022'
}

In [ ]:
import pandas as pd
from IPython.display import display


def get_data(abs_params):
    """
    Get data from the Cenus Bureau API
    
    Parameters:
    - variables: Comma-separated string of variable names
    - geography: Geographic level (default: national level)
    - api_key: Your Census Bureau API key
    """
    base_url = "https://api.census.gov/data"
    year = abs_params['year']
    dataset = abs_params['dataset_base']
    variables = abs_params['variables']
    geography = abs_params['geography']
    api_key = abs_params['api_key']
    
    # Construct the full URL
    url = f"{base_url}/{year}/{dataset}?get={variables}&for={geography}"
    print(url)
    
    if api_key:
        url += f"&key={api_key}"
        
    response = requests.get(url)
    
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"Failed to retrieve data. Status code: {response}")

def get_all_business_code_table(df, naics_code, naics_colum):
    n = len(naics_code)
    full_list = []
    for i in range(n):

        naics_temp = naics_code[:i]
        naics_table = df[(df[naics_colum]==naics_temp)]

        if naics_table.shape[0]:
            full_list.append(naics_table)
        
    full_df = pd.concat(full_list)
    return  full_df

In [ ]:
nonemp = pd.DataFrame(get_data(nonemp_params))
nonemp.columns = nonemp.iloc[0]
nonemp = nonemp.iloc[1:]
nonemp[(nonemp['NAICS2022']==NAICS)]

In [ ]:
all_bc_nonemp = get_all_business_code_table(nonemp, NAICS, 'NAICS2022')
display(all_bc_nonemp)

In [ ]:
cbp = pd.DataFrame(get_data(cbp_params))
cbp.columns = cbp.iloc[0]
cbp = cbp.iloc[1:]
cbp[(cbp['NAICS2017']==NAICS)]

In [ ]:
ecnbasic = pd.DataFrame(get_data(ecnbasic_params))
ecnbasic.columns = ecnbasic.iloc[0]
ecnbasic = ecnbasic.iloc[1:]
ecnbasic[(ecnbasic['NAICS2022']==NAICS)]

In [ ]:
all_bc_ecnbasic = get_all_business_code_table(nonemp, NAICS, 'NAICS2022')
display(all_bc_ecnbasic)

## 2. Research Potential Locations:
- **Demographics:** Research the demographics of potential areas to ensure they align with your target market
- **Competition:** Analyze the competitive landscape in each area to understand the level of competition
- **Infrastructure:** Assess the availability of essential infrastructure, such as 
    - transportation
    - utilities
    - internet access
- **Zoning Laws:** Understand local zoning regulations and restrictions to ensure compliance
- **Local Resources:** Explore local government census data or tools like WIGeoLocation for further research  
- **Accessibility:** Evaluate the accessibility of the location for both customers and employees
- **Traffic:** Consider traffic patterns and parking availability, especially if foot traffic is important 

## 3. Evaluate and Compare:  
- **Visit Potential Locations:** Conduct site visits to assess the suitability of each location firsthand
- **Cost Analysis:** Compare the costs of different 
    - locations
    - including rent
    - utilities
    - taxes  
- **Pros and Cons:** Create a list of pros and cons for each potential location to help make a decision
- **Long-Term Growth Potential:** Consider the long-term growth potential of the area and its ability to support your business  

## 4. Consider Legal and Institutional Factors: 
- **Business Licenses:** Research the necessary business licenses and permits required for your business type and location. 
- **Local Regulations:** Understand any local regulations or restrictions that may affect your business operations.
- **Government Incentives:** Explore any local or state government incentives or programs that may be available for businesses in specific areas.